In [ ]:
import json, urllib.request, urllib.error, csv

In [ ]:
ws_list = list()
with open('ws.csv', newline='') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=';')
  for row in spamreader:
    ws_list.append(row[0])
ws_list[:10]

['керамзитобетонные блоки в Новосибирске',
 'керамзитоблоки Новосибирск',
 'фбс Новосибирск',
 'бетон с доставкой Новосибирск',
 'керамзитобетонные блоки Новосибирск',
 'керамзитные блоки Новосибирск',
 'плиты перекрытия Новосибирск',
 'купить бетон Новосибирск',
 'купить керамзитобетонные блоки в Новосибирск',
 'купить керамзитоблоки Новосибирск']

In [ ]:
quries_dict = {}
with open('queries.csv', newline='') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=';')
  for row in spamreader:
    if row[2] == 'search_query' or row[2] in ws_list: 
      print(row[2])
      continue
    quries_dict[row[2]] = row[6]


In [ ]:
for query, lr in quries_dict.items():
  print(query, lr)

In [ ]:
#https://github.com/igor-kantor/Yandex.Wordstat-parser/blob/master/wsparser.py

class WordstatParser:
    def __init__(self, url, token, username):
        self.url = url
        self.token = token
        self.username = username
        
    def getClientUnits(self):
        data = {
            'method': 'GetClientsUnits',
            'token': self.token,
            'param': [self.username]
        }
        data = json.dumps(data, ensure_ascii=False).encode('utf8')
        request = urllib.request.urlopen(self.url,data) 
        response = json.loads(request.read().decode('utf8'))
        return response

    def createReport(self, phrases, geo = []):
        data = {
            'method': 'CreateNewWordstatReport',
            'token': self.token,
            'param': {
                'Phrases': phrases,
                'GeoID': geo
                }
        }
        data = json.dumps(data, ensure_ascii=False).encode('utf8')
        request = urllib.request.urlopen(self.url,data) 
        response = json.loads(request.read().decode('utf8'))
        return response
    
    def getReportList (self):
        data = {
            'method': 'GetWordstatReportList',
            'token': self.token    
        }
        data = json.dumps(data, ensure_ascii=False).encode('utf8')
        request = urllib.request.urlopen(self.url,data) 
        response = json.loads(request.read().decode('utf8'))
        return response
        
    def readReport (self, reportID):
        data = {
            'method': 'GetWordstatReport',
            'token': self.token,
            'param': reportID
        }
        data = json.dumps(data, ensure_ascii=False).encode('utf8')
        request = urllib.request.urlopen(self.url,data) 
        response = json.loads(request.read().decode('utf8'))
        return response
    
    def deleteReport (self, reportID):
        data = {
            'method': 'DeleteWordstatReport',
            'token': self.token,
            'param': reportID
        }
        data = json.dumps(data, ensure_ascii=False).encode('utf8')
        request = urllib.request.urlopen(self.url,data) 
        response = json.loads(request.read().decode('utf8'))
        return response

    def saveReportToTxt (self, report, rightCol):
        
        #print(report)
        phrases = open('phrases_left.txt', 'a')
        shows = open('shows_left.txt', 'a')
        for i in range(len(report['data'])):
            res = report['data'][i]['SearchedWith'][0] # - это пишем в файл {'Phrase': 'фбс уфа', 'Shows': 86}
            with open('ws.csv', 'a', newline='') as csvfile:
              spamwriter = csv.writer(csvfile, delimiter=';')
              spamwriter.writerow([res['Phrase'], res['Shows']])


            for j in report['data'][i]['SearchedWith']:
                phraseToReport = str(j['Phrase'])
                phrases.write(phraseToReport+'\n')
                showsToReport = str(j['Shows'])
                shows.write(showsToReport+'\n')
        phrases.close()
        shows.close()
        if rightCol == True:
            phrases = open('phrases_right.txt', 'a')
            shows = open('shows_right.txt', 'a')
            for i in range(len(report['data'])):
                for j in report['data'][i]['SearchedAlso']:
                    phraseToReport = str(j['Phrase'])
                    phrases.write(phraseToReport+'\n')
                    showsToReport = str(j['Shows'])
                    shows.write(showsToReport+'\n')
            phrases.close()
            shows.close()


In [ ]:
import time

##################################################################################
# Вводим исходные данные для парсинга
##################################################################################

# Задаем URL адрес API Яндекс.Директа
# Адрес песочницы: url ='https://api-sandbox.direct.yandex.ru/v4/json/' 
# Адрес полного доступа: url ='https://api.direct.yandex.ru/v4/json/'
url ='https://api-sandbox.direct.yandex.ru/v4/json/' 
# Указываем свой токен на доступ к API Яндекс.Директ
token = '*****'
# Указываем логин своей учетной записи от Яндекс.Директ
userName = '******'

In [ ]:
for query, lr in quries_dict.items():
  time.sleep(1)
  print(query, lr)
  # Пишем список общих минус-слов, как в примере (со знаком "-")
  minusWords = [
      ]

  # Пишем список фраз, по которым будем парсить    
  phrases = [
      query
      ]

  # Указываем регион, при необходимости (можно оставить пустым)        
  geo = [lr]

  ##################################################################################
  # Код скрипта парсинга
  ##################################################################################

  # Добавляем минус-слова ко всем фразам
  data = []
  for i in range(len(phrases)):
      data.append(phrases[i])
      for j in range(len(minusWords)):
          data[i] += ' '+minusWords[j]

  # Создаем парсер
  parser = WordstatParser(url, token, userName)

  try:
      # Запрашиваем кол-во оставшихся баллов Яндекс.Директ API
      units = parser.getClientUnits()
      if 'data' in units:
          print ('>>> Баллов осталось: ', units['data'][0]['UnitsRest'])
      else:
          raise Exception('Не удалось получить баллы', units)

      # Отправляем запрос на создание нового отчета на сервере Яндекс.Директ
      response = parser.createReport(data, geo)
      if 'data' in response:
          reportID = response['data']
          print ('>>> Создается отчет с ID = ', reportID)
      else:
          raise Exception('Не удалось создать отчет', response)
          
      # Проверяем список отчетов на сервере. Должен появиться новый. Ожидаем его готовности
      reportList = parser.getReportList()
      if 'data' in reportList:
          lastReport = reportList['data'][len(reportList['data'])-1]
          i = 0
          while lastReport['StatusReport'] != 'Done':
              print ('>>> Подготовка отчета, ждите ... ('+str(i)+')')
              time.sleep(2)
              reportList = parser.getReportList()
              lastReport = reportList['data'][len(reportList['data'])-1]
              i+=1
          print ('>>> Отчет ID = ', lastReport['ReportID'], ' получен!')
      else:
          raise Exception('Не удалось прочитать список отчетов', reportList)

      # Читаем отчет
      report = parser.readReport(reportID)
      if 'data' in report:
          # Сохраняем результаты парсинга в файлы (отдельно фразы, отдельно частотности). 
          # Если rightCol == True, будет сохраняться правая колонка Яндекс.Вордстат (в дополнение к левой)
          parser.saveReportToTxt(report, True)
          print ('>>> Результаты парсига успешно сохранены в файлы!')
      else:
          raise Exception('Не удалось прочитать отчет', report)
      
      # Удаляем на сервере Яндекс.Директ новый отчет (он больше не нужен)
      report = parser.deleteReport(reportID)
      if 'data' in report:
          print ('>>> Отчет с ID = ', reportID, ' успешно удален с сервера Яндекс.Директ')
      else:
          raise Exception('Не удалось удалить отчет', report)
      
      print ('>>> Все готово!')
      
  # Все готово! Ищем файлы "phrases_lef.txt" и "shows_left.txt" с резльтатами парсинга в директории этого скрипта
  except Exception as e:
      with open('logs.csv', 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=';')
        spamwriter.writerow([query, lr])
      print ('>>> Поймано исключение:', e)
  reportList = parser.getReportList()
  try:
    if len(reportList['data']) > 0:
      for row in reportList['data']:
        report = parser.deleteReport(row['ReportID'])
        if 'data' in report:
            print ('>>> Отчет с ID = ', row['ReportID'], ' успешно удален с сервера Яндекс.Директ')
        else:
            raise Exception('Не удалось удалить отчет', report)
  except:
    print('no data to delete')

Выходные данные были обрезаны до нескольких последних строк (5000).
>>> Баллов осталось:  31750
>>> Создается отчет с ID =  4836952
>>> Подготовка отчета, ждите ... (0)
>>> Отчет ID =  4836952  получен!
>>> Результаты парсига успешно сохранены в файлы!
>>> Отчет с ID =  4836952  успешно удален с сервера Яндекс.Директ
>>> Все готово!
купить плиты перекрытия Екатеринбург 54
>>> Баллов осталось:  31740
>>> Создается отчет с ID =  4836953
>>> Подготовка отчета, ждите ... (0)
>>> Подготовка отчета, ждите ... (1)
>>> Отчет ID =  4836953  получен!
>>> Результаты парсига успешно сохранены в файлы!
>>> Отчет с ID =  4836953  успешно удален с сервера Яндекс.Директ
>>> Все готово!
керамзитобетонные блоки в Казани 43
>>> Баллов осталось:  31730
>>> Создается отчет с ID =  4836955
>>> Подготовка отчета, ждите ... (0)
>>> Подготовка отчета, ждите ... (1)
>>> Отчет ID =  4836955  получен!
>>> Результаты парсига успешно сохранены в файлы!
>>> Отчет с ID =  4836955  успешно удален с сервера Яндекс.Дире

In [ ]:
from datetime import datetime
datetime.now()

datetime.datetime(2020, 12, 21, 19, 26, 46, 37310)

In [ ]:
parser = WordstatParser(url, token, userName)

In [ ]:
reportList = parser.getReportList()
print('reports:', len(reportList['data']))
for row in reportList['data']:
  report = parser.deleteReport(row['ReportID'])
  if 'data' in report:
      print ('>>> Отчет с ID = ', row['ReportID'], ' успешно удален с сервера Яндекс.Директ')
  else:
      raise Exception('Не удалось удалить отчет', report)

reports: 0


In [ ]:
reportList

{'data': [{'ReportID': 4618297, 'StatusReport': 'Done'},
  {'ReportID': 4618299, 'StatusReport': 'Done'},
  {'ReportID': 4618301, 'StatusReport': 'Done'},
  {'ReportID': 4618304, 'StatusReport': 'Done'},
  {'ReportID': 4618312, 'StatusReport': 'Done'},
  {'ReportID': 4618316, 'StatusReport': 'Done'},
  {'ReportID': 4618319, 'StatusReport': 'Done'},
  {'ReportID': 4618321, 'StatusReport': 'Done'},
  {'ReportID': 4618326, 'StatusReport': 'Done'},
  {'ReportID': 4618337, 'StatusReport': 'Done'},
  {'ReportID': 4618341, 'StatusReport': 'Done'},
  {'ReportID': 4618349, 'StatusReport': 'Done'},
  {'ReportID': 4618355, 'StatusReport': 'Done'},
  {'ReportID': 4618360, 'StatusReport': 'Done'},
  {'ReportID': 4618363, 'StatusReport': 'Done'},
  {'ReportID': 4618366, 'StatusReport': 'Done'},
  {'ReportID': 4618367, 'StatusReport': 'Done'},
  {'ReportID': 4618368, 'StatusReport': 'Done'},
  {'ReportID': 4618369, 'StatusReport': 'Done'},
  {'ReportID': 4618375, 'StatusReport': 'Done'},
  {'ReportID